##### THIS FILE IS USED TO CALCULATE CLV #####

In [1]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import math
import os
from dateutil.relativedelta import relativedelta
from sklearn.cluster import KMeans
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_tree
# Logistic Regresison 
import statsmodels
from  statsmodels.discrete.discrete_model import Logit
import sklearn
from sklearn.linear_model import LogisticRegression

In [2]:
# This is used to block warning messages
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
# Or ignore specific warnings by category (e.g., FutureWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)
# This is used to edit image
from IPython.display import Image
from IPython.core.display import HTML 

In [3]:
data_df = pd.read_csv(r'C:\Users\Admin\RBAC\Kmean_4.csv', index_col=0)
df_ori = pd.read_csv(r'C:\Users\Admin\RBAC\OriDataNew.csv').iloc[:,1:]

In [4]:
df_ori['TransactionDate'] = pd.to_datetime(df_ori['TransactionDate'])
df_ori['Year_Month'] = df_ori['TransactionDate'].apply(lambda x: x.strftime('%Y-%m'))

In [5]:
df_merge = df_ori.merge(data_df,on='CustomerID',how='left')
df_merge.columns

Index(['BillID', 'Channel', 'OrderFrom', 'TransactionDate', 'SalesAmount',
       'CustomerID', 'CustomerGender_x', 'VoucherStatus', 'Province_x',
       'Year_Month', 'mean_days_between_purchase', 'max_days_between_purchase',
       'TotalValue', 'CountOrder', 'ValuePerOrder', 'Channel_Delivery',
       'Channel_TakeAway', 'Channel_DineIn', 'OrderFrom_Store',
       'OrderFrom_Web', 'OrderFrom_CallCenter', 'OrderFrom_App',
       'CustomerGender_y', 'Province_y', 'First_Order_Day', 'Last_Order_Day',
       'LifeTime', 'Days_between_last_cur', 'Cluster'],
      dtype='object')

In [6]:
np.random.rand()

0.3013496593332412

In [12]:
clv_data = df_merge.groupby(['CustomerID','Year_Month','Cluster'],as_index=False).agg(
                                                                            mean_lifetime = ('LifeTime','mean'),
                                                                            mean_order_value = ('ValuePerOrder', 'mean'),
                                                                            num_of_order_per_month = ('BillID','count'),
                                                                            )
clv_data['mean_lifetime_month'] = clv_data['mean_lifetime'].apply(lambda x: x/30 if x>30 else 1) 
clv_data['num_of_order_per_month_noise'] = clv_data['num_of_order_per_month'] + np.random.randint(1,3)*clv_data['num_of_order_per_month']
clv_data['CLV'] = (clv_data['num_of_order_per_month'] * clv_data['mean_order_value']) * clv_data['mean_lifetime_month']
clv_data

,CustomerID,Year_Month,Cluster,mean_lifetime,mean_order_value,num_of_order_per_month,mean_lifetime_month,num_of_order_per_month_noise,CLV
0,0,2021-12,0,0.0,411663.000000,1,1.000000,3,4.116630e+05
1,1,2022-03,0,0.0,105503.000000,1,1.000000,3,1.055030e+05
2,8,2022-07,2,240.0,970848.000000,1,8.000000,3,7.766784e+06
3,8,2023-03,2,240.0,970848.000000,1,8.000000,3,7.766784e+06
4,14,2023-05,0,0.0,267607.000000,1,1.000000,3,2.676070e+05
...,...,...,...,...,...,...,...,...,...
1230811,2174072,2022-10,2,82.0,317872.666667,1,2.733333,3,8.688520e+05
1230812,2174072,2022-12,2,82.0,317872.666667,1,2.733333,3,8.688520e+05
1230813,2174073,2023-02,0,0.0,514503.000000,1,1.000000,3,5.145030e+05
1230814,2174075,2022-12,0,0.0,262489.000000,1,1.000000,3,2.624890e+05


In [13]:
clv_data_gr_cluster = clv_data.groupby(['Cluster','Year_Month'],as_index=False).agg({
    'mean_lifetime_month':'mean',
    'mean_order_value':'mean',
    'num_of_order_per_month':'mean',
    'num_of_order_per_month_noise':'mean',
    'CLV':'mean'
})


clv_data_gr_cluster.to_csv(r'clv_data_gr_cluster2.csv')

In [14]:
clv_data.to_csv(r'clv_data.csv')